In [1]:
### create pickle file with SAX dimer features
## PAA time window = 0.05 #0.25s  # 0.5, 0.1s

import pandas as pd
import numpy as np
import os
import pickle
#import re
import regex as re
import matplotlib.pyplot as plt

In [2]:
## create SAX string representation & dimer features

# STEPS
# for each epoch:
# 1. normalize the data over a window (epoch size)
# 2. convert to PAA - take small segments & represent by the average over the segment
# 3. choose alphabet size & the levels/ threshold for discretisation
# 4. convert to string representation
# 5. vector representation - numbers of different dimers

# discretise over 0.25S = 0.25 * 100 = 25 points; reduces to 120 points
#               0.5s => 50 points; reduces to 60 points
#               0.1 s = 0.1 * 100 = 10 points -> 300 data points
#               0.05s = 5 points -> 600 data points


### 1. normalise the data, 1 epoch (30s, i.e. 3000 data points) at a time ###
# if the SD is very small (less than a threshold), do not normalize the data
def z_norm(y):
    sd = y.std()
    if sd < 0.01:
        y = y
    else:
        y =(y - y.mean())/sd
    
    return y



#### 2. convert to PAA - take a window & represent by the average over the window ####
# discretise over 0.25S = 0.25 * 100 = 25 points  -> results in 120 data points
#               0.5s = 50 points   -> results in 60 data points
#               0.1 s = 0.1 * 100 = 10 points -> 300 data points
#               0.05s = 5 points -> 600 data points
#time_window = 0.5
def get_paa(y, time_window):
    # convert time_window from sec to no. of data points
    time_window = time_window * 100  # 100 is sampling freq
    
    paa = []
    for i in range(0, int(len(y)/time_window)): # len(y)/time_window gives the no. of steps
        # take a segment of data
        arr = y[(i * int(time_window)):((i+1) * int(time_window))]
        # convert to mean & append to list
        paa.append(arr.mean())
    return paa



#### 3. choose alphabet size & the levels/ threshold for discretisation ####
### set this up as a dictionary to choose different alphabet sizes  ###
# let alphabet size = 4;
# cuts: [-0.67, 0, 0.67]



#### 4. convert to string  ####
def get_sax_string(paa):
    sax = []
    
    for i in range(0,len(paa)):
        if paa[i] <= -0.67:
            alpha = 'a'
        elif (paa[i] > -0.67 and paa[i] <=0):
            alpha = 'b'
        elif (paa[i] > 0 and paa[i] <=0.67):
            alpha = 'c'
        else:
            alpha = 'd'
        sax.append(alpha)
    
    # convert sax from list to string format
    sax_string = ''.join(sax)
    return sax_string



#### 5. vector representation - numbers of different dimers ####
# different types of dimers
# aa ab ac ad
# ba bb bc bd
# ca cb cc cd
# da db dc dd
### ways to count ###
# sax_string.count(dim_alpha[i]) => 'bbb' : counts bb as 1  => FIX THIS
# len(re.findall(dim_alpha[i], sax_string)) : counts bb as 1

###  set up a list of the dimers possible  ###
dim_alpha = ['aa','ab','ac','ad','ba','bb','bc','bd','ca','cb','cc','cd','da','db','dc','dd']

def get_sax_dimer_array(sax_string):
    dim_arr = []
    
    for i in range(0, len(dim_alpha)):
        dim_arr.append(len(re.findall(dim_alpha[i], sax_string, overlapped=True)))
    #https://stackoverflow.com/questions/2970520/string-count-with-overlapping-occurrences
    
    #print(dim_arr)
    #ncols = 4
    #im = np.reshape(dim_arr,(-1,ncols)) #x.reshape(1754, 1754)
    #plt.gray()
    #plt.imshow(im)
    return dim_arr



def run_all(y, time_window):
    y = z_norm(y)
    #time_window = 0.5
    paa = get_paa(y, time_window)
    sax_string = get_sax_string(paa)
    sax_feature = get_sax_dimer_array(sax_string)
    ## added the following 2 lines to make the data fraction
    total = sum(sax_feature)
    sax_feature = [x/total for x in sax_feature]
    return sax_feature

In [3]:
#time_window = 0.1 #0.5 #0.25

def get_PSD_from_pickle(pid,day,x):
    df = []
    eeg_class = []
    epochnum = []

    
    for k in list(x['wake'].keys()):
        label = 'wake'  #seg = k.split('e')[0]
        epoch = k
        if "s1e" not in epoch:  ## leave out the first long wake segment
            y = x['wake'][k]
            df.append(run_all(y, time_window))  #x['wake'][k])
            eeg_class.append(label)
            epochnum.append(k)

    for k in list(x['sleep_stage_1'].keys()):
        label = 'sleep_stage_1'   #seg = k.split('e')[0]
        epoch = k
        y = x['sleep_stage_1'][k]
        df.append(run_all(y, time_window))  #x['sleep_stage_1'][k])
        eeg_class.append(label)
        epochnum.append(k)
            
    for k in list(x['sleep_stage_2'].keys()):
        label = 'sleep_stage_2'   #seg = k.split('e')[0]
        epoch = k
        y = x['sleep_stage_2'][k]
        df.append(run_all(y, time_window))  #x['sleep_stage_2'][k])
        eeg_class.append(label)
        epochnum.append(k)

    #print(df)    
    df = pd.DataFrame(df)  ## also, add subject ID
    print(df)
    df['pID'] = pid
    df['day'] = day
    eeg_class = pd.DataFrame(eeg_class, columns=['class_label'])
    epochnum = pd.DataFrame(epochnum, columns=['epoch'])
    
    df = pd.concat([df,epochnum,eeg_class],axis=1)
    
    return df

In [4]:
file_location='C:/Users/DAR9KOR/Desktop/data/HEALTHCARE/EEG/data/eeg_epochs_dec12_2018/raw_data' #C:\\Users\\DAR9KOR\\Desktop\\data\\HEALTHCARE\\EEG\\data\\eeg_epochs_dec12_2018'
os.chdir('C:/Users/DAR9KOR/Desktop/data/HEALTHCARE/EEG/data/eeg_epochs_dec12_2018/raw_data') #'/home/intern_eyecare/Desktop/EEG/raw_data') #C:\\Users\\DAR9KOR\\Desktop\\data\\HEALTHCARE\\EEG\\data\\eeg_epochs_dec12_2018')

files=os.listdir(file_location)
file_list = [f for f in files if f.endswith('.pickle')]   # .txt  # .csv

In [5]:
time_window = 0.1 #0.025  #0.05 #0.1 #0.5 #0.25 
dfull = pd.DataFrame()

for file_val in file_list:
    #print(file_val)
    patient = file_val[2:5]
    day = file_val[5:6]
    
    with open(file_val, 'rb') as f:
        print(file_val)
        x = pickle.load(f)      
        dat = get_PSD_from_pickle(patient,day,x)
    f.close()
        
    dfull = pd.concat([dfull, dat],ignore_index=True)

SC4001E0-PSG-EEG.pickle
            0         1         2         3         4         5         6   \
0     0.093960  0.043624  0.026846  0.003356  0.046980  0.117450  0.107383   
1     0.080537  0.023490  0.000000  0.000000  0.020134  0.288591  0.097315   
2     0.120805  0.023490  0.000000  0.000000  0.020134  0.244966  0.083893   
3     0.163880  0.053512  0.023411  0.003344  0.066890  0.163880  0.033445   
4     0.120401  0.036789  0.033445  0.020067  0.053512  0.140468  0.036789   
5     0.133779  0.066890  0.003344  0.003344  0.063545  0.173913  0.050167   
6     0.167224  0.050167  0.013378  0.010033  0.056856  0.130435  0.060201   
7     0.130435  0.056856  0.020067  0.006689  0.066890  0.163880  0.056856   
8     0.120401  0.033445  0.000000  0.000000  0.026756  0.193980  0.093645   
9     0.133779  0.066890  0.006689  0.000000  0.056856  0.103679  0.090301   
10    0.053512  0.066890  0.030100  0.010033  0.066890  0.140468  0.100334   
11    0.056856  0.070234  0.026756  0.02

            0         1         2         3         4         5         6   \
0     0.120805  0.050336  0.013423  0.000000  0.046980  0.130872  0.083893   
1     0.057047  0.060403  0.016779  0.006711  0.057047  0.164430  0.124161   
2     0.046980  0.073826  0.026846  0.010067  0.053691  0.124161  0.093960   
3     0.033557  0.036913  0.020134  0.003356  0.046980  0.231544  0.117450   
4     0.167224  0.050167  0.003344  0.000000  0.053512  0.217391  0.053512   
5     0.143813  0.036789  0.003344  0.000000  0.036789  0.227425  0.073579   
6     0.107023  0.043478  0.000000  0.000000  0.040134  0.214047  0.096990   
7     0.123746  0.030100  0.010033  0.000000  0.036789  0.257525  0.080268   
8     0.187291  0.050167  0.003344  0.000000  0.046823  0.130435  0.060201   
9     0.147157  0.050167  0.000000  0.003344  0.043478  0.167224  0.080268   
10    0.040134  0.060201  0.030100  0.013378  0.063545  0.153846  0.096990   
11    0.053512  0.070234  0.046823  0.013378  0.083612  0.093645

            0         1         2         3         4         5         6   \
0     0.151007  0.053691  0.030201  0.000000  0.060403  0.127517  0.036913   
1     0.147157  0.053512  0.013378  0.000000  0.053512  0.133779  0.076923   
2     0.167785  0.060403  0.013423  0.000000  0.063758  0.154362  0.046980   
3     0.157191  0.053512  0.006689  0.000000  0.050167  0.157191  0.053512   
4     0.177258  0.050167  0.003344  0.000000  0.046823  0.143813  0.056856   
5     0.123746  0.060201  0.003344  0.000000  0.056856  0.117057  0.073579   
6     0.160535  0.046823  0.020067  0.000000  0.050167  0.113712  0.056856   
7     0.127090  0.040134  0.013378  0.000000  0.030100  0.170569  0.060201   
8     0.150502  0.050167  0.000000  0.006689  0.050167  0.177258  0.060201   
9     0.157191  0.050167  0.003344  0.000000  0.046823  0.133779  0.066890   
10    0.193980  0.070234  0.000000  0.000000  0.060201  0.107023  0.063545   
11    0.164430  0.057047  0.003356  0.000000  0.046980  0.124161

            0         1         2         3         4         5         6   \
0     0.171141  0.046980  0.003356  0.003356  0.040268  0.147651  0.063758   
1     0.117057  0.056856  0.023411  0.000000  0.073579  0.173913  0.043478   
2     0.133779  0.043478  0.030100  0.016722  0.053512  0.086957  0.050167   
3     0.114094  0.040268  0.013423  0.003356  0.040268  0.194631  0.083893   
4     0.090604  0.036913  0.006711  0.003356  0.033557  0.164430  0.100671   
5     0.056856  0.020067  0.003344  0.000000  0.016722  0.287625  0.090301   
6     0.113712  0.020067  0.003344  0.003344  0.013378  0.254181  0.083612   
7     0.086957  0.016722  0.000000  0.000000  0.013378  0.357860  0.063545   
8     0.110368  0.026756  0.010033  0.003344  0.036789  0.210702  0.063545   
9     0.167224  0.073579  0.023411  0.000000  0.060201  0.083612  0.046823   
10    0.153846  0.030100  0.016722  0.000000  0.036789  0.197324  0.056856   
11    0.180602  0.053512  0.020067  0.003344  0.060201  0.066890

            0         1         2         3         4         5         6   \
0     0.164430  0.050336  0.000000  0.000000  0.046980  0.194631  0.060403   
1     0.087248  0.046980  0.010067  0.006711  0.046980  0.228188  0.097315   
2     0.140940  0.050336  0.033557  0.003356  0.063758  0.117450  0.080537   
3     0.147651  0.053691  0.003356  0.003356  0.050336  0.181208  0.070470   
4     0.157718  0.033557  0.006711  0.000000  0.026846  0.161074  0.077181   
5     0.151007  0.070470  0.010067  0.000000  0.077181  0.140940  0.060403   
6     0.096990  0.020067  0.000000  0.000000  0.020067  0.274247  0.063545   
7     0.100671  0.050336  0.010067  0.000000  0.057047  0.184564  0.073826   
8     0.107023  0.043478  0.003344  0.000000  0.050167  0.224080  0.076923   
9     0.153846  0.033445  0.000000  0.000000  0.030100  0.227425  0.060201   
10    0.150502  0.046823  0.006689  0.003344  0.043478  0.197324  0.050167   
11    0.093645  0.030100  0.010033  0.020067  0.050167  0.244147

...

            0         1         2         3         4         5         6   \
0     0.110368  0.083612  0.020067  0.000000  0.050167  0.113712  0.080268   
1     0.120401  0.070234  0.013378  0.020067  0.056856  0.103679  0.090301   
2     0.100334  0.050167  0.016722  0.003344  0.046823  0.143813  0.073579   
3     0.124161  0.053691  0.013423  0.000000  0.046980  0.177852  0.077181   
4     0.107023  0.050167  0.020067  0.003344  0.050167  0.123746  0.096990   
5     0.096990  0.053512  0.013378  0.000000  0.030100  0.130435  0.107023   
6     0.066890  0.050167  0.013378  0.010033  0.046823  0.227425  0.133779   
7     0.083612  0.043478  0.016722  0.006689  0.043478  0.150502  0.107023   
8     0.063545  0.043478  0.026756  0.010033  0.060201  0.187291  0.110368   
9     0.107023  0.056856  0.006689  0.016722  0.046823  0.167224  0.107023   
10    0.093645  0.056856  0.006689  0.013378  0.040134  0.180602  0.120401   
11    0.100334  0.043478  0.020067  0.013378  0.056856  0.173913

            0         1         2         3         4         5         6   \
0     0.130435  0.086957  0.013378  0.003344  0.090301  0.090301  0.060201   
1     0.153846  0.036789  0.003344  0.000000  0.030100  0.214047  0.066890   
2     0.163880  0.070234  0.026756  0.013378  0.083612  0.113712  0.036789   
3     0.127090  0.040134  0.013378  0.010033  0.053512  0.217391  0.070234   
4     0.046980  0.053691  0.013423  0.000000  0.050336  0.218121  0.104027   
5     0.060201  0.080268  0.010033  0.000000  0.060201  0.170569  0.093645   
6     0.130872  0.046980  0.023490  0.000000  0.033557  0.110738  0.093960   
7     0.090301  0.056856  0.006689  0.006689  0.060201  0.127090  0.096990   
8     0.117450  0.060403  0.013423  0.000000  0.053691  0.120805  0.114094   
9     0.073579  0.040134  0.010033  0.000000  0.033445  0.177258  0.120401   
10    0.090604  0.046980  0.013423  0.000000  0.050336  0.174497  0.107383   
11    0.096990  0.040134  0.003344  0.003344  0.026756  0.260870

            0         1         2         3         4         5         6   \
0     0.087248  0.060403  0.010067  0.003356  0.057047  0.147651  0.114094   
1     0.046980  0.067114  0.023490  0.003356  0.073826  0.184564  0.107383   
2     0.076923  0.050167  0.026756  0.010033  0.050167  0.096990  0.110368   
3     0.093645  0.053512  0.020067  0.000000  0.050167  0.123746  0.086957   
4     0.087248  0.053691  0.013423  0.003356  0.053691  0.137584  0.104027   
5     0.140468  0.046823  0.016722  0.000000  0.036789  0.133779  0.080268   
6     0.130435  0.060201  0.010033  0.000000  0.050167  0.110368  0.073579   
7     0.113712  0.056856  0.016722  0.000000  0.053512  0.137124  0.073579   
8     0.053691  0.050336  0.026846  0.003356  0.043624  0.184564  0.114094   
9     0.063758  0.023490  0.013423  0.010067  0.030201  0.214765  0.127517   
10    0.080537  0.046980  0.033557  0.000000  0.043624  0.140940  0.110738   
11    0.110368  0.043478  0.006689  0.003344  0.043478  0.210702

            0         1         2         3         4         5         6   \
0     0.114094  0.060403  0.000000  0.000000  0.053691  0.154362  0.104027   
1     0.090604  0.043624  0.013423  0.000000  0.053691  0.164430  0.093960   
2     0.140468  0.043478  0.003344  0.000000  0.036789  0.177258  0.107023   
3     0.100334  0.033445  0.003344  0.003344  0.030100  0.170569  0.086957   
4     0.124161  0.060403  0.016779  0.006711  0.053691  0.137584  0.097315   
5     0.107023  0.060201  0.006689  0.000000  0.046823  0.147157  0.090301   
6     0.043478  0.050167  0.013378  0.006689  0.046823  0.210702  0.127090   
7     0.127517  0.050336  0.006711  0.003356  0.040268  0.130872  0.087248   
8     0.076923  0.043478  0.013378  0.003344  0.050167  0.204013  0.113712   
9     0.053691  0.050336  0.010067  0.000000  0.046980  0.221477  0.120805   
10    0.070234  0.053512  0.020067  0.000000  0.053512  0.153846  0.117057   
11    0.060403  0.067114  0.006711  0.000000  0.060403  0.214765

           0         1         2         3         4         5         6   \
0    0.096990  0.043478  0.003344  0.000000  0.043478  0.187291  0.103679   
1    0.060201  0.060201  0.023411  0.000000  0.066890  0.193980  0.107023   
2    0.140468  0.053512  0.020067  0.000000  0.056856  0.137124  0.073579   
3    0.113712  0.053512  0.010033  0.003344  0.060201  0.200669  0.090301   
4    0.147157  0.053512  0.003344  0.003344  0.046823  0.133779  0.100334   
5    0.123746  0.060201  0.016722  0.000000  0.063545  0.173913  0.083612   
6    0.147157  0.040134  0.006689  0.000000  0.040134  0.150502  0.070234   
7    0.063545  0.083612  0.013378  0.006689  0.076923  0.204013  0.096990   
8    0.100334  0.060201  0.016722  0.003344  0.063545  0.167224  0.090301   
9    0.130435  0.050167  0.006689  0.003344  0.050167  0.167224  0.090301   
10   0.133779  0.043478  0.020067  0.000000  0.056856  0.150502  0.096990   
11   0.117057  0.046823  0.006689  0.006689  0.046823  0.183946  0.110368   

In [6]:
## save data
pickle.dump(dfull, open('Frac_sax_dimer_0.1s_rm_1st_wake.pkl','wb'))

In [7]:
dfull.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,pID,day,epoch,class_label
0,0.093960,0.043624,0.026846,0.003356,0.046980,0.117450,0.107383,0.026846,0.026846,0.104027,0.134228,0.070470,0.003356,0.033557,0.067114,0.093960,400,1,s2e1,wake
1,0.080537,0.023490,0.000000,0.000000,0.020134,0.288591,0.097315,0.003356,0.003356,0.093960,0.218121,0.030201,0.000000,0.000000,0.033557,0.107383,400,1,s3e1,wake
2,0.120805,0.023490,0.000000,0.000000,0.020134,0.244966,0.083893,0.000000,0.003356,0.077181,0.278523,0.016779,0.000000,0.000000,0.016779,0.114094,400,1,s4e1,wake
3,0.163880,0.053512,0.023411,0.003344,0.066890,0.163880,0.033445,0.016722,0.013378,0.053512,0.096990,0.066890,0.000000,0.010033,0.076923,0.157191,400,1,s5e1,wake
4,0.120401,0.036789,0.033445,0.020067,0.053512,0.140468,0.036789,0.013378,0.036789,0.063545,0.197324,0.040134,0.000000,0.006689,0.066890,0.133779,400,1,s5e2,wake


In [8]:
dfull.groupby('class_label').size()

class_label
sleep_stage_1     2804
sleep_stage_2    17799
wake             37566
dtype: int64

In [10]:
dfull.groupby(['pID','class_label']).size()

pID  class_label  
400  sleep_stage_1     117
     sleep_stage_2     623
     wake             1992
401  sleep_stage_1     201
     sleep_stage_2    1222
     wake             2319
402  sleep_stage_1     278
     sleep_stage_2     947
     wake             2292
403  sleep_stage_1     106
     sleep_stage_2     885
     wake             2173
404  sleep_stage_1     303
     sleep_stage_2    1134
     wake             1421
405  sleep_stage_1     158
     sleep_stage_2     833
     wake             1668
406  sleep_stage_1     146
     sleep_stage_2     824
     wake             2294
407  sleep_stage_1     173
     sleep_stage_2     795
     wake             2289
408  sleep_stage_1     107
     sleep_stage_2     591
     wake             2061
409  sleep_stage_1     100
     sleep_stage_2    1073
     wake             1116
410  sleep_stage_1     182
     sleep_stage_2    1278
     wake             2037
411  sleep_stage_1      31
     sleep_stage_2     898
     wake             1813
412  slee